In [186]:
import numpy as np
import random
import math
import time
import copy
import pandas as pd
import operator
from collections import defaultdict
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [187]:
class Node(object):
    def __init__(self, data = None, next_node = None, prev_node = None):
        self.data = data
        self.next_node =  next_node
        self.prev_node = prev_node

class DoublyLinkedList(object):
    def __init__(self, head = None):
        self.head = head
    
    def traverse(self):
        current_node = self.head
        while current_node != None:
            print(current_node.data)
            current_node = current_node.next_node
    
    def get_size(self):
        count = 0
        current_node = self.head
        while current_node != None:
            count += 1
            current_node = current_node.next_node
        return count
            
    def append(self, data,df):
        new_node = Node(data)
        df.loc[data,'Node'] = new_node
        current_node = self.head
        new_node.next_node = current_node
        new_node.prev_node = None
        if current_node != None:
            current_node.prev_node = new_node
        self.head = new_node
    
    def insert_end(self, data):
        new_node = Node(data)
        new_node.next = None
        if self.head == None:
            new_node.prev_node = None
            self.head = new_node
        return
    
        first_node = self.head
        while first_node.next_node:
            first_node = first_node.next_node
        first_node.next_node = new_node
        new_node.prev_node = first_node
    
    
    def delete(self, data):
        current_node = self.head
        while current_node != None:
            if current_node.data == data and current_node == self.head:
                if not current_node.next_node:
                    current_node = None
                    self.head = None
                    return
                else:
                    q = current_node.next_node
                    current_node.next_node = None
                    q.prev_node = None
                    current_node = None
                    self.head = q
                    return
            
            elif current_node.data == data:
                #print("hello")
                if current_node.next_node != None:
                    p = current_node.prev_node
                    q = current_node.next_node
                    p.next_node = q
                    q.prev_node = p
                    current_node.next_node = None
                    current_node.prev_node = None
                    current_node = None
                    return
                else:
                    #print("bye")
                    p = current_node.prev_node
                    p.next_node = None
                    current_node.prev_node = None
                    current = None
                    return
            current_node =  current_node.next_node
    
    def delete_node(self,node_to_delete):
        if(node_to_delete.prev_node == None and node_to_delete.next_node == None):
            self.head = None
            del node_to_delete
        elif(node_to_delete.prev_node == None and node_to_delete.next_node != None):
            q = node_to_delete.next_node
            self.head = q
            q.prev_node = None
            node_to_delete = None
        elif(node_to_delete.prev_node != None and node_to_delete.next_node == None):
            p = node_to_delete.prev_node
            p.next_node = None
            node_to_delete = None
        else:
            p = node_to_delete.prev_node
            q = node_to_delete.next_node
            p.next_node = q
            q.prev_node = p
            node_to_delete = None

class Bucket(object):
    def __init__(self, value):
        self.data = DoublyLinkedList()

class Bucket_Arrays(object):
    def __init__(self, maxdegree):
        self.bucket_range = maxdegree
        self.left_buckets = np.array([Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)])
        self.right_buckets = np.array([Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)])
        self.max_gain_left = -maxdegree
        self.max_gain_right = -maxdegree
        self.left_size = 0
        self.right_size = 0
        self.fixed = 0

In [188]:
def calculate_num_cuts(df):
    connected_edges = df[df['Partition']==0]['ID connected vertices'].values
    outside_partition = list(df[df['Partition']==1].index)
    connected_edges = [item in outside_partition for sublist in connected_edges for item in sublist]
    num_cuts = np.sum(connected_edges)
    return num_cuts

def initialize_gain_buckets(df, buckets):
    max_degree = 16
    for i in range(len(df)):
        connected_edges = df['ID connected vertices'][i]
        partition_value = df['Partition'][i]
        gain = 0
        for j in connected_edges:
            connected_partition = df['Partition'][j]
            if(partition_value != connected_partition):
                gain += 1
            else:
                gain -= 1
        df.loc[i,'Gain'] = gain
        
        if partition_value == 0:
            buckets.left_buckets[gain+max_degree].data.append(i,df)
            buckets.left_size += 1
            if(buckets.max_gain_left < gain):
                buckets.max_gain_left = gain
        else:
            buckets.right_buckets[gain+max_degree].data.append(i,df)
            buckets.right_size += 1
            if(buckets.max_gain_right < gain):
                buckets.max_gain_right = gain
    return df,buckets

###
def update_df_buckets(df,buckets,vertex_max_gain, gain_update,max_degree):

    partition_value = df.loc[vertex_max_gain,'Partition']
    df.loc[vertex_max_gain,'Partition'] = int(not(partition_value))
    node_to_delete = df.loc[vertex_max_gain,'Node']
    if df.loc[vertex_max_gain,'Partition'] == 1:
        buckets.left_buckets[gain_update+max_degree].data.delete_node(node_to_delete)
        df.loc[vertex_max_gain,'Node'] = 0
        buckets.left_size -= 1
        while(buckets.left_buckets[buckets.max_gain_left+max_degree].data.head == None and buckets.max_gain_left>-max_degree):
            buckets.max_gain_left -= 1
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain,max_degree,1)
        
    else:
        buckets.right_buckets[gain_update+max_degree].data.delete_node(node_to_delete)
        df.loc[vertex_max_gain,'Node'] = 0
        buckets.right_size -= 1
        while(buckets.right_buckets[buckets.max_gain_right+max_degree].data.head == None and buckets.max_gain_right>-max_degree):
            buckets.max_gain_right -= 1
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain,max_degree,0)
    return df, buckets

def re_calculate_gain(df, buckets, vertex_max_gain, max_degree, changed_partition):
    vertices_to_update = df.loc[vertex_max_gain]['ID connected vertices']
    for i in vertices_to_update:
        if(df.loc[i]['Fixed'] == 0):
            current_gain = df.loc[i]['Gain']
            current_partition = df.iloc[i]['Partition']
            node_to_delete = df.loc[i,'Node']
            if current_partition == 1:
                buckets.right_buckets[current_gain+max_degree].data.delete_node(node_to_delete)
                df.loc[i,'Node'] = 0
                buckets.right_size -= 1
                while(buckets.right_buckets[buckets.max_gain_right+max_degree].data.head == None and buckets.max_gain_right>-max_degree):
                    buckets.max_gain_right -= 1
            else:
                buckets.left_buckets[current_gain+max_degree].data.delete_node(node_to_delete)
                df.loc[i,'Node'] = 0
                buckets.left_size -= 1
                while(buckets.left_buckets[buckets.max_gain_left+max_degree].data.head == None and buckets.max_gain_left>-max_degree):
                    buckets.max_gain_left -= 1
        
            partition_value = df['Partition'][i]
            if (partition_value == changed_partition):
                new_gain = current_gain - 2
            else:
                new_gain = current_gain + 2
            df.loc[i,'Gain'] = new_gain
            if(partition_value == 1): 
                buckets.right_buckets[new_gain+max_degree].data.append(i,df)
                buckets.right_size += 1
                if(buckets.max_gain_right < new_gain):
                    buckets.max_gain_right = new_gain
            else:
                buckets.left_buckets[new_gain+max_degree].data.append(i,df)
                buckets.left_size += 1
                if(buckets.max_gain_left < new_gain):
                    buckets.max_gain_left = new_gain
    df.loc[vertex_max_gain,'Fixed'] = 1
    buckets.fixed += 1
    #df.loc[vertex_max_gain,'Gain'] = -999
    return df, buckets

def initialise_data(partitioning):
    # Loading the single planar graph of 500 vertices
    data = defaultdict(list)
    for line in open("Graph500.txt"):
        split_line=line.split()
        ID_vertex = int(split_line[0])
        num_connected_vertices  = int(split_line[2])
        ID_connected_vertices = [int(i)-1 for i in split_line[3:]]
        if (ID_vertex) not in data.keys():
            data[ID_vertex].append(0)
            data[ID_vertex].append(0)
            data[ID_vertex].append(ID_connected_vertices)
            data[ID_vertex].append(0)
            data[ID_vertex].append(0)
    data_frame = pd.DataFrame(data.values(),columns = ['Gain', 'Fixed','ID connected vertices', 'Partition','Node'])

    num_vertices = len(data_frame)
    if(partitioning is None):
        partition = random.sample(range(0,num_vertices),250)
        data_frame.loc[partition,'Partition'] = 1
    else:
        data_frame["Partition"] = partitioning
    return data_frame

def FM_one_pass(df, current_num_cuts):
    num_cuts = current_num_cuts
    num_vertices = 500
    min_cuts = num_cuts
    max_degree = 16
    buckets = Bucket_Arrays(max_degree)
    df,buckets = initialize_gain_buckets(df, buckets)
    save_partition = copy.deepcopy(df['Partition'].values)
    while(buckets.fixed < num_vertices):
        if(buckets.left_size >= buckets.right_size):
            gain_update = buckets.max_gain_left
            vertex_max_gain = buckets.left_buckets[gain_update+max_degree].data.head.data
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain,gain_update,max_degree)
            
        else:
            gain_update = buckets.max_gain_right
            vertex_max_gain = buckets.right_buckets[gain_update+max_degree].data.head.data
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain,gain_update,max_degree)
            
        num_cuts = num_cuts - gain_update
        if(num_cuts < min_cuts and buckets.left_size == buckets.right_size):
            save_partition = copy.deepcopy(df['Partition'].values)
            min_cuts = num_cuts
    return min_cuts, save_partition

In [190]:
flag = 0
total_passes = 0
total_time = 0
best_local_optimum = math.inf
optimal_partition = None
    
df = initialise_data(optimal_partition)
current_num_cuts = calculate_num_cuts(df)
start = time.time()
local_optimum, optimal_partition = FM_one_pass(df,current_num_cuts)
total_passes += 1
end = time.time()
elpased_time = end - start
print(elpased_time)

2.8316903114318848


In [179]:
local_optimum

45

In [4]:
def FM_one_run(max_passes):
    flag = 0
    total_passes = 0
    total_time = 0
    best_local_optimum = math.inf
    optimal_partition = None
    while(True):
        df = initialise_data(optimal_partition)
        current_num_cuts = calculate_num_cuts(df)
        start = time.time()
        local_optimum, optimal_partition = FM_one_pass(df,current_num_cuts)
        total_passes += 1
        end = time.time()
        elpased_time = end - start
        total_time += elpased_time

        if(local_optimum < best_local_optimum and total_passes <= max_passes):
            best_local_optimum = local_optimum
            flag = 0
        else:
            flag = 1
        if(flag == 1):
            break
    converged_local_optimum = best_local_optimum
    return [converged_local_optimum, total_time, total_passes] 

In [16]:
node_to_delete.prev_node.next_node

AttributeError: 'NoneType' object has no attribute 'next_node'

In [5]:
# FM_baseline
max_passes = 10000
total_runs = 25
run_data_frame = pd.DataFrame(columns = ['Coverged local optima','Time(s)','Number of passes'])
for i in range(0,25): 
    print(i)
    observations = FM_one_run(max_passes)
    run_data_frame.loc[len(run_data_frame)] = observations
#run_data_frame.to_csv(r'FM_baseline.csv', index = False)
#np.savetxt(r'FM_baseline_median.txt', run_data_frame.median(), fmt='%f')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [7]:
run_data_frame.to_csv(r'FM_baseline_linear_time.csv', index = False)